In [1]:
import numpy as np
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
import os

In [8]:
def make_map(image_folder, filename, rows, cols):
    tiles = []

    for i in range(rows):
        col_tiles = []
        for j in range(cols):
            file = f"{filename}_{i}_{j}.png"
            img = Image.open(os.path.join(image_folder, file))
            col_tiles.append(img)
        tiles.append(col_tiles)

    tile_width, tile_height = tiles[0][0].size

    full_width = cols * tile_width 
    full_height = rows * tile_height 

    full_image = Image.new('RGB', (full_width, full_height))

    y_offset = 0
    for col_list in tiles:
        x_offset = 0
        for sing_img in col_list:
            full_image.paste(sing_img, (x_offset, y_offset))
            x_offset += tile_width
        y_offset += tile_height

    full_image_np = np.array(full_image)

    plt.figure(figsize=(10, 10))
    plt.imshow(full_image_np)
    plt.axis('off')
    plt.show()

In [9]:
def calc_flood_mean_value(flood_image, mask):
    water_mask = mask > 128
    water_pixels = flood_image[water_mask]
    return np.mean(water_pixels, axis=0)

def flood_mean_value(flood_images, masks, row, col):
    flood_means = []
    for i in range(row+1):
        for j in range(col+1):
            flood_means.append(calc_flood_mean_value(flood_images[i][j], masks[i][j]))

    if row < len(flood_images)-1:
        flood_means.append(calc_flood_mean_value(flood_images[i+1][j], masks[i+1][j]))
        if col > 1:
            flood_means.append(calc_flood_mean_value(flood_images[i+1][j-1], masks[i+1][j-1]))
    
    if col < len(flood_images[0])-1:
        flood_means.append(calc_flood_mean_value(flood_images[i][j+1], masks[i][j+1]))

        if row > 1:
            flood_means.append(calc_flood_mean_value(flood_images[i-1][j+1], masks[i-1][j+1]))

    if col < len(flood_images[0])-1 and row < len(flood_images)-1:
        flood_means.append(calc_flood_mean_value(flood_images[i+1][j+1], masks[i+1][j+1]))

    return np.nanmean(np.array(flood_means), axis=0)

In [10]:
def improve_mask_with_old_diff_and_only_close_mean_flood_value(sentinel2_dir, natural_water_masks_dir, flood_masks_dir, old_dir, save_dir, filename, rows, cols):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    non_flood_images = [[io.imread(f"./{old_dir}/{filename}_{i}_{j}.png") for j in range(cols)] for i in range(rows)]
    flood_images = [[io.imread(f'./{sentinel2_dir}/{filename}_{i}_{j}.png') for j in range(cols)] for i in range(rows)]
    flood_masks = [[io.imread(f'./{flood_masks_dir}/{filename}_{i}_{j}.png') for j in range(cols)] for i in range(rows)]
    natural_water_masks = [[io.imread(f'./{natural_water_masks_dir}/{filename}_{i}_{j}.png') for j in range(cols)] for i in range(rows)]

    for i in range(rows):
        for j in range(cols):
            flood_image = flood_images[i][j]
            non_flood_image = non_flood_images[i][j]
            mask = flood_masks[i][j]
            natural_water_mask = natural_water_masks[i][j]

            natural_water_mask = natural_water_mask > 128
            mask[natural_water_mask] = 0

            flood_mean_rgb = flood_mean_value(flood_images, flood_masks, i, j)
            
            water_mask = mask > 128
            
            flood_mask = np.all((flood_image >= flood_mean_rgb*0.85) & (flood_image <= flood_mean_rgb*1.15), axis=-1)
            non_flood_mask = np.all((non_flood_image >= flood_mean_rgb*0.85) & (non_flood_image <= flood_mean_rgb*1.15), axis=-1)

            inverse_flood_mask = np.logical_not(flood_mask)

            highlighted_flood_image = np.copy(flood_image)
            
            highlighted_flood_image[flood_mask] = [255, 255, 255]
            highlighted_flood_image[inverse_flood_mask] = [0, 0, 0]
            highlighted_flood_image[non_flood_mask] = [0, 0, 0]
            highlighted_flood_image[water_mask] = [255, 255, 255]

            io.imsave(f'./{save_dir}/{filename}_{i}_{j}.png', highlighted_flood_image)
            